In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen as uReq
import requests
import lxml
import pandas as pd
from pandas import DataFrame
import numpy as np

print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O 'india_data.json' https://data.gov.in/node/128290/download
print('Data downloaded!')

Data downloaded!


In [3]:
my_url='https://helperindia.com/list-delhi-metro-stations-gps-information/'

In [4]:
r=requests.get(my_url)


In [5]:
# Parse the htlm with Soup
page=bsoup(r.text,"html.parser")
page

<!DOCTYPE html>

<html lang="en-US" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://helperindia.com/xmlrpc.php" rel="pingback"/>
<title>Delhi Metro Station List with GPS Information - Helper India</title>
<script type="text/javascript">function theChampLoadEvent(e){var t=window.onload;if(typeof window.onload!="function"){window.onload=e}else{window.onload=function(){t();e()}}}</script>
<script type="text/javascript">var theChampCloseIconPath = 'https://helperindia.com/wp-content/plugins/super-socializer/images/close.png';var heateorSsSDKBlockedMsg = `Your browser is blocking some features of this website. Please follow the instructions at <a href=\'http://support.heateor.com/browser-blocking-social-features/\' target=\'_blank\' style=\'color:#33a9d8\'>http://support.heateor.com/browser-blocking-social-features/</a> to unblock these

In [6]:
# finding number of rows of the table


In [7]:
# the first few lines indicate that there is indeed the table we need to find that table
gtable=page.table
gtable

<table width="477">
<tbody>
<tr>
<td width="303"><strong>Station Name</strong></td>
<td width="87"><strong>Latitude</strong></td>
<td width="87"><strong>Longitude</strong></td>
</tr>
<tr>
<td>Adarsh Nagar Metro Station</td>
<td>28.716499</td>
<td>77.170505</td>
</tr>
<tr>
<td>AIIMS Metro Station</td>
<td>28.566828</td>
<td>77.20812</td>
</tr>
<tr>
<td>Akshardham Metro Station</td>
<td>28.618002</td>
<td>77.279372</td>
</tr>
<tr>
<td>Anand Vihar Metro Station</td>
<td>28.646989</td>
<td>77.316019</td>
</tr>
<tr>
<td>Arjan Garh Metro Station</td>
<td>28.480793</td>
<td>77.125775</td>
</tr>
<tr>
<td>Ashok Park Main Metro Station</td>
<td>28.6716</td>
<td>77.155077</td>
</tr>
<tr>
<td>Azadpur Metro Station</td>
<td>28.707011</td>
<td>77.180542</td>
</tr>
<tr>
<td>Badarpur Border Metro Station</td>
<td>28.493215</td>
<td>77.303001</td>
</tr>
<tr>
<td>Badkhal Mor Metro Station</td>
<td>28.422814</td>
<td>77.310278</td>
</tr>
<tr>
<td>Barakhambha Road Metro Station</td>
<td>28.629907</td>
<td

In [8]:
results=gtable.find_all('tr')
nrows=len(results)
nrows

155

In [9]:
header=results[0].text.split()
header

['Station', 'Name', 'Latitude', 'Longitude']

In [10]:
# iteration loop to harvest all records

records =[]
n=1
while n < nrows :
    Station=results[n].text.split('\n')[1]
    Latitude=results[n].text.split('\n')[2]
    Longitude=results[n].text.split('\n')[3]
    
    records.append((Station, Latitude,Longitude))
    n=n+1

df=pd.DataFrame(records, columns=['Station Name', 'Latitude', 'Longitude'])


df.head()

In [11]:
df.head()

,Station Name,Latitude,Longitude
0,Adarsh Nagar Metro Station,28.716499,77.170505
1,AIIMS Metro Station,28.566828,77.20812
2,Akshardham Metro Station,28.618002,77.279372
3,Anand Vihar Metro Station,28.646989,77.316019
4,Arjan Garh Metro Station,28.480793,77.125775


In [12]:
df.shape

(154, 3)

In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Libraries imported.


In [46]:
# exploring delhi metro stations same like manhattan

CLIENT_ID = '5ODENMW5WTRY0XIU3FIHNXX5DXOZFEFTU3DVYLP4S3IQA413' 
CLIENT_SECRET = '1A4KCRSMAL2IRNVVGW5DKGOAWS4JGDRLXDEJTUPWDXKYW1QE'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID , 
            CLIENT_SECRET , 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
#  client id etc for foursquare
CLIENT_ID = '5ODENMW5WTRY0XIU3FIHNXX5DXOZFEFTU3DVYLP4S3IQA413' # your Foursquare ID
CLIENT_SECRET = '1A4KCRSMAL2IRNVVGW5DKGOAWS4JGDRLXDEJTUPWDXKYW1QE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5ODENMW5WTRY0XIU3FIHNXX5DXOZFEFTU3DVYLP4S3IQA413
CLIENT_SECRET:1A4KCRSMAL2IRNVVGW5DKGOAWS4JGDRLXDEJTUPWDXKYW1QE


In [17]:
delhi_venues = getNearbyVenues(names=df['Station Name'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Adarsh Nagar Metro Station
AIIMS Metro Station
Akshardham Metro Station
Anand Vihar Metro Station
Arjan Garh Metro Station
Ashok Park Main Metro Station
Azadpur Metro Station
Badarpur Border Metro Station
Badkhal Mor Metro Station
Barakhambha Road Metro Station
Bata Chowk Metro Station
Botanical Garden Metro Station
Central Secretariat Metro Station
Chandni Chowk Metro Station
Chawri Bazar Metro Station
Chhatarpur Metro Station
Civil Lines Metro Station
Delhi Aerocity Metro Station
Delhi Gate Metro Station
Dhaula Kuan Metro Station
Dilshad Garden Metro Station
Dwarka Metro Station
Dwarka Mor Metro Station
Dwarka Sector 10 Metro Station
Dwarka Sector 11 Metro Station
Dwarka Sector 12 Metro Station
Dwarka Sector 13 Metro Station
Dwarka Sector 14 Metro Station
Dwarka Sector 21 Metro Station
Dwarka Sector 8 Metro Station
Dwarka Sector 9 Metro Station
Escorts Mujesar Metro Station
Ghitorni Metro Station
Golf Course Metro Station
Govind Puri Metro Station
Green Park Metro Station
GTB Nagar M

In [18]:
# making a map of delhi  with these  stations

In [19]:
# latitude longidtude of delhi from google
latitude = 28.7041
longitude= 77.1025

In [20]:
# create map of Delhi using latitude and longitude values above:
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

 
    
map_delhi

In [21]:
# start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [22]:
# exploring delhi same like manhattan
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
#  client id etc for foursquare
#  venues by neighbourhood
CLIENT_ID = '5ODENMW5WTRY0XIU3FIHNXX5DXOZFEFTU3DVYLP4S3IQA413' # your Foursquare ID
CLIENT_SECRET = '1A4KCRSMAL2IRNVVGW5DKGOAWS4JGDRLXDEJTUPWDXKYW1QE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5ODENMW5WTRY0XIU3FIHNXX5DXOZFEFTU3DVYLP4S3IQA413
CLIENT_SECRET:1A4KCRSMAL2IRNVVGW5DKGOAWS4JGDRLXDEJTUPWDXKYW1QE


In [25]:
delhi_venues = getNearbyVenues(names=df['Station Name'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Adarsh Nagar Metro Station
AIIMS Metro Station
Akshardham Metro Station
Anand Vihar Metro Station
Arjan Garh Metro Station
Ashok Park Main Metro Station
Azadpur Metro Station
Badarpur Border Metro Station
Badkhal Mor Metro Station
Barakhambha Road Metro Station
Bata Chowk Metro Station
Botanical Garden Metro Station
Central Secretariat Metro Station
Chandni Chowk Metro Station
Chawri Bazar Metro Station
Chhatarpur Metro Station
Civil Lines Metro Station
Delhi Aerocity Metro Station
Delhi Gate Metro Station
Dhaula Kuan Metro Station
Dilshad Garden Metro Station
Dwarka Metro Station
Dwarka Mor Metro Station
Dwarka Sector 10 Metro Station
Dwarka Sector 11 Metro Station
Dwarka Sector 12 Metro Station
Dwarka Sector 13 Metro Station
Dwarka Sector 14 Metro Station
Dwarka Sector 21 Metro Station
Dwarka Sector 8 Metro Station
Dwarka Sector 9 Metro Station
Escorts Mujesar Metro Station
Ghitorni Metro Station
Golf Course Metro Station
Govind Puri Metro Station
Green Park Metro Station
GTB Nagar M

In [26]:
print(delhi_venues.shape)
delhi_venues.head()

(1171, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar Metro Station,28.716499,77.170505,Vishyavidyalaya Metro Station@Entry gate #1 n ...,28.715596,77.170981,Train Station
1,Adarsh Nagar Metro Station,28.716499,77.170505,Adarsh Nagar Metro Station,28.716598,77.170436,Light Rail Station
2,Adarsh Nagar Metro Station,28.716499,77.170505,My Idea Store,28.717487,77.170922,Mobile Phone Shop
3,Adarsh Nagar Metro Station,28.716499,77.170505,Pahalwan Vaishno Dhaba,28.715881,77.173565,North Indian Restaurant
4,Adarsh Nagar Metro Station,28.716499,77.170505,Yummy Tummy Restaurant,28.718103,77.175133,Fast Food Restaurant


In [27]:
# check how many venues were returned for each station


In [28]:
delhi_venues.groupby('Neighbourhood').count()


,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
AIIMS Metro Station,5,5,5,5,5,5
Adarsh Nagar Metro Station,5,5,5,5,5,5
Akshardham Metro Station,6,6,6,6,6,6
Anand Vihar Metro Station,8,8,8,8,8,8
Arjan Garh Metro Station,5,5,5,5,5,5
Ashok Park Main Metro Station,1,1,1,1,1,1
Azadpur Metro Station,3,3,3,3,3,3
Badarpur Border Metro Station,3,3,3,3,3,3
Badkhal Mor Metro Station,1,1,1,1,1,1


In [29]:
# number of unique categories


In [30]:
print('There are {} uniques categories.'.format(len(delhi_venues['Venue Category'].unique())))


There are 174 uniques categories.


In [31]:
# one hot encoding for analysing each neighbourhood
delhi_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Neighbourhood'] = delhi_venues['Neighbourhood'] 

# move neighborhood column to the first column
cols=list(delhi_onehot.columns.values)
cols.pop(cols.index('Neighbourhood'))
delhi_onehot=delhi_onehot[['Neighbourhood']+cols]



In [32]:
delhi_onehot.shape


(1171, 175)

In [33]:
# grouping neighbourhoods by taking the mean of the frequency of occurrence of each category
delhi_grouped = delhi_onehot.groupby('Neighbourhood').mean().reset_index()
delhi_grouped

,Neighbourhood,ATM,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Center,Business Service,Cafeteria,Café,Campground,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Duty-free Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gift Shop,Go Kart Track,Golf Course,Government Building,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Hindu Temple,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Leather Goods Store,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Mosque,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,North Indian Restaurant,Office,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Rest Area,Restaurant,River,Road,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Tea Room,Temple,Thai Restaurant,Theater,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store,Yoga Studio
0,AIIMS Metro Station,0.000000,0.000000,0.000000,0.2,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.200000,0.0,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Adarsh Nagar Metro Station,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000

In [34]:
# list of each Metro Station with 5 most common venues
num_top_venues = 5

for Stat in delhi_grouped['Neighbourhood']:
    print("----"+Stat+"----")
    temp = delhi_grouped[delhi_grouped['Neighbourhood'] == Stat].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AIIMS Metro Station----
                venue  freq
0                 Spa   0.2
1  Airport Food Court   0.2
2         Snack Place   0.2
3      Clothing Store   0.2
4  Light Rail Station   0.2


----Adarsh Nagar Metro Station----
                     venue  freq
0  North Indian Restaurant   0.2
1        Mobile Phone Shop   0.2
2            Train Station   0.2
3       Light Rail Station   0.2
4     Fast Food Restaurant   0.2


----Akshardham Metro Station----
                     venue  freq
0            Metro Station  0.17
1        Convenience Store  0.17
2             Dessert Shop  0.17
3  North Indian Restaurant  0.17
4             Hindu Temple  0.17


----Anand Vihar Metro Station----
                venue  freq
0           Multiplex  0.25
1               Hotel  0.12
2       Shopping Mall  0.12
3  Light Rail Station  0.12
4                 Gym  0.12


----Arjan Garh Metro Station----
                    venue  freq
0                   Hotel   0.4
1      Light Rail Station   0.2
2

In [35]:
# put in pandas dataframe
# sort top venues in decdeing order 

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
# new dataframe with top 10 venues of each neighbourhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = delhi_grouped['Neighbourhood']

for ind in np.arange(delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AIIMS Metro Station,Spa,Airport Food Court,Light Rail Station,Clothing Store,Snack Place,Event Space,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
1,Adarsh Nagar Metro Station,North Indian Restaurant,Light Rail Station,Train Station,Fast Food Restaurant,Mobile Phone Shop,Electronics Store,Food,Flea Market,Farmers Market,Farm
2,Akshardham Metro Station,North Indian Restaurant,Dessert Shop,Metro Station,Restaurant,Convenience Store,Hindu Temple,Deli / Bodega,Department Store,Food,Flea Market
3,Anand Vihar Metro Station,Multiplex,Café,Hotel,Shopping Mall,Indian Restaurant,Light Rail Station,Gym,Event Space,Food,Flea Market
4,Arjan Garh Metro Station,Hotel,Yoga Studio,Furniture / Home Store,Light Rail Station,Event Space,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
5,Ashok Park Main Metro Station,Train Station,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
6,Azadpur Metro Station,Park,Indian Restaurant,Bus Station,Yoga Studio,Event Space,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
7,Badarpur Border Metro Station,ATM,Train Station,IT Services,Event Space,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
8,Badkhal Mor Metro Station,IT Services,Yoga Studio,Event Space,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Farm
9,Barakhambha Road Metro Station,Indian Restaurant,Bar,Historic Site,Asian Restaurant,Portuguese Restaurant,Fast Food Restaurant,Salon / Barbershop,Bistro,Café,Beer Garden


In [54]:
# new dataframe with top 10 venues of each neighbourhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = delhi_grouped['Neighbourhood']

for ind in np.arange(delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AIIMS Metro Station,Spa,Airport Food Court,Light Rail Station,Clothing Store,Snack Place,Event Space,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
1,Adarsh Nagar Metro Station,North Indian Restaurant,Light Rail Station,Train Station,Fast Food Restaurant,Mobile Phone Shop,Electronics Store,Food,Flea Market,Farmers Market,Farm
2,Akshardham Metro Station,North Indian Restaurant,Dessert Shop,Metro Station,Restaurant,Convenience Store,Hindu Temple,Deli / Bodega,Department Store,Food,Flea Market
3,Anand Vihar Metro Station,Multiplex,Café,Hotel,Shopping Mall,Indian Restaurant,Light Rail Station,Gym,Event Space,Food,Flea Market
4,Arjan Garh Metro Station,Hotel,Yoga Studio,Furniture / Home Store,Light Rail Station,Event Space,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
5,Ashok Park Main Metro Station,Train Station,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
6,Azadpur Metro Station,Park,Indian Restaurant,Bus Station,Yoga Studio,Event Space,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
7,Badarpur Border Metro Station,ATM,Train Station,IT Services,Event Space,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
8,Badkhal Mor Metro Station,IT Services,Yoga Studio,Event Space,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Farm
9,Barakhambha Road Metro Station,Indian Restaurant,Bar,Historic Site,Asian Restaurant,Portuguese Restaurant,Fast Food Restaurant,Salon / Barbershop,Bistro,Café,Beer Garden


In [38]:
df=df.set_index('Station Name')


In [39]:
df = df.drop(['Vidhan Sabha Metro Station'], axis=0)

In [40]:
df.head()

,Latitude,Longitude
Station Name,,
Adarsh Nagar Metro Station,28.716499,77.170505
AIIMS Metro Station,28.566828,77.20812
Akshardham Metro Station,28.618002,77.279372
Anand Vihar Metro Station,28.646989,77.316019
Arjan Garh Metro Station,28.480793,77.125775


In [52]:
df.shape

(153, 2)

In [42]:
# run k mean cluster let us keep number of clusters as 5

# set number of clusters
kclusters = 5

delhi_grouped_clustering = delhi_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 2, 2, 2, 3, 2, 3, 2, 2], dtype=int32)

In [53]:
neighborhoods_venues_sorted.shape

(151, 11)

In [44]:
df.head()

,Latitude,Longitude
Station Name,,
Adarsh Nagar Metro Station,28.716499,77.170505
AIIMS Metro Station,28.566828,77.20812
Akshardham Metro Station,28.618002,77.279372
Anand Vihar Metro Station,28.646989,77.316019
Arjan Garh Metro Station,28.480793,77.125775


In [50]:
# new database with these clusters as well as top 10 venues in each neighbourhood

delhi_merged = df

# add clustering labels
delhi_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
delhi_merged = delhi_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Station Name')

delhi_merged.head() 
# check the last columns!


ValueError: Length of values does not match length of index

In [51]:
# create map for cluster visualisation
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon,poi, cluster in zip(delhi_merged['Latitude'], delhi_merged['Longitude'], delhi_merged['1st Most Common Venue'], delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[(cluster)]
    ).add_to(map_clusters)
       
map_clusters

KeyError: '1st Most Common Venue'

# create map for cluster visualisation
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon,poi, cluster in zip(delhi_merged['Latitude'], delhi_merged['Longitude'], delhi_merged['1st Most Common Venue'], delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters